In [1]:
import numpy as np
import matplotlib.pylab as plot
from astropy.io import ascii,fits
from scipy import interpolate
import grb_catalogs_copy
from BurstCube.LocSim.Detector import *
from BurstCube.LocSim.Spacecraft import *
from astropy.coordinates import SkyCoord
from astropy import units as u
from scipy.optimize import curve_fit

In [2]:
# read in GBM GRB catalog (first 9 years)
gbm=fits.open('gbmgrbcat_copy.fits')
gbm=gbm[1].data

In [3]:
## code to use when reading in GBM effective area in order to get data into the desired format
def getGBMdata(gbmfile=None):
    """Reads the GBM NaI effective area file and returns a numpy array
    with two columns ``energy`` and ``aeff``.
    Parameters
    ----------
    gbmfile : string
       Name of file that contains the GBM data.
    Returns
    ----------
    gbmdata : array 
    numpy array with two columns ``energy`` and ``aeff``
    """

   # from numpy import genfromtxt
    #from os import path

    #gbmfile = path.expandvars(gbmfile)
    
    return np.genfromtxt(gbmfile,skip_header=2,names=('energy', 'aeff'))

In [4]:
## bit of useful code for interpolating in log space
def loginterpol(x,y,x1):

    f=interpolate.interp1d(np.log10(x),np.log10(y),bounds_error=False,fill_value="extrapolate",kind='linear')
    y1=10**f(np.log10(x1))

    return y1

def loginterpol2d(x,y,z,x1,y1):

    wz=np.where(z==0)[0]
    zz=z
    zz[wz]=1.
    f=interpolate.interp2d(x,y,np.log10(zz),bounds_error=False,fill_value="extrapolate",kind='linear')
    z1=10**f(x1,y1)

In [5]:
## generate random positions on the sky with equal area probability
def random_sky(n=1):

    u=np.random.rand(n)
    v=np.random.rand(n)

    phi=2*np.pi*u
    theta=np.arccos(2*v-1.)

    dec=-np.degrees(theta-np.pi/2.)
    ra=np.degrees(np.pi*2-phi)

    return ra,dec

In [6]:
## read in the GBM Aeff
aeff_gbm = getGBMdata('/home/alyson/NASA/Simulation/BurstCube/Users/ajoens/gbm_effective_area.dat')
print(aeff_gbm)

[(8.10939400e+00,   6.043869 ) (8.27917900e+00,   7.1733994)
 (8.54091550e+00,   9.051345 ) (8.71935400e+00,  11.009052 )
 (9.09046600e+00,  13.555708 ) (9.47675300e+00,  17.315489 )
 (9.77528500e+00,  23.22683  ) (1.05165080e+01,  30.780512 )
 (1.16751795e+01,  43.367252 ) (1.26926560e+01,  58.89598  )
 (1.50095340e+01,  80.0014   ) (1.83216400e+01,  97.346825 )
 (2.26054130e+01, 106.105095 ) (3.13079800e+01, 120.00855  )
 (3.26618730e+01, 102.37214  ) (4.11476300e+01, 123.06248  )
 (8.95676700e+01, 132.68503  ) (1.45272660e+02, 131.2247   )
 (3.44349330e+02,  75.82555  ) (1.07285790e+03,  42.783394 )
 (2.76511250e+03,  30.818037 ) (9.45941300e+03,  32.459995 )]


In [7]:
## separate GBM short & long GRBs
w=np.where(gbm['FLUX_1024']>0)
gbm=gbm[w]
s=np.where(gbm['T90'] <= 2.)[0]
l=np.where(gbm['T90'] > 2.)[0]
m=np.where(gbm['PFLX_BEST_FITTING_MODEL'][s] == ' ')

In [8]:
#Integrating the best fit spectrum for each GRB in the energy range of 50-300 KeV. 
#This will give us the photon flux in units of ph/cm^2/s. Currently only doing this for GBM and will then add in BurstCube
# the parts below I have commented out are what I believe are for BC and currently I am focusing on GBM
mo=gbm['PFLX_BEST_FITTING_MODEL'][s]
#f=np.zeros([len(s),nsims]) # produces an array of zeros with the given shape and type
pf=np.zeros(len(s))
outE=np.logspace(np.log10(50),np.log10(300),20) # returns numbers spaced evenly on a log scale
for i in range(len(s)):
    #for j in range(nsims):
        #E=np.array(eng[w[j]+1:w[j+1]+1])
        #AeffBC=loginterpol(E,aeffs['aeff'][w[j]+1:w[j+1]+1],outE)
        AeffGBM=loginterpol(aeff_gbm['energy'],aeff_gbm['aeff'],outE)#eng[w[j]+1:w[j+1]+1])
        #Aratio=(AeffBC/AeffGBM)
        # not sure what *grb_catalogs_copy.pl(outE,gbm['PFLX_PLAW_INDEX'][s[i]] is and why we need it. I think we only need the model photon flux times the aeffGBM and we want it integrated over the energy range provided in outE
        # this should give us an array of the maximum observed photon flux for GBM
        if mo[i]=='PFLX_PLAW':
            pf[i]=np.trapz(gbm['PFLX_PLAW_AMPL'][s[i]]*grb_catalogs_copy.pl(outE,gbm['PFLX_PLAW_INDEX'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_PLAW_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_COMP':
            pf[i]=np.trapz(gbm['PFLX_COMP_AMPL'][s[i]]*grb_catalogs_copy.comp(outE,gbm['PFLX_COMP_INDEX'][s[i]],gbm['PFLX_COMP_EPEAK'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_COMP_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_BAND':
            pf[i]=np.trapz(gbm['PFLX_BAND_AMPL'][s[i]]*grb_catalogs_copy.band(outE,gbm['PFLX_BAND_ALPHA'][s[i]],gbm['PFLX_BAND_EPEAK'][s[i]],gbm['PFLX_BAND_BETA'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_BAND_PHTFLUX'][s[i]]
        if mo[i]=='PFLX_SBPL':
            pf[i]=np.trapz(gbm['PFLX_SBPL_AMPL'][s[i]]*grb_catalogs_copy.sbpl(outE,gbm['PFLX_SBPL_INDX1'][s[i]],gbm['PFLX_SBPL_BRKEN'][s[i]],gbm['PFLX_SBPL_INDX2'][s[i]])*AeffGBM,outE)
            #pf[i]=gbm['PFLX_SBPL_PHTFLUX'][s[i]]
            
#print(pf)

In [9]:
# we want to solve for the number of detected counts. We need to create two more arrays- 1 for exposure and the other for interval.
#Interval is given in the GBM catalog however we need to know exposure. This should be found from the exposure map for each burst.
#this all together will give us the number of source photons

## setup GBM
gbm_pointings = {'01': ('45:54:0','20:36:0'),
            '02': ('45:6:0','45:18:0'),
            '03': ('58:24:0','90:12:0'),
            '04': ('314:54:0','45:12:0'),
            '05': ('303:12:0','90:18:0'),
            '06': ('3:24:0','89:48:0'),
            '07': ('224:54:0','20:24:0'),
            '08': ('224:36:0','46:12:0'),
            '09': ('236:36:0','90:0:0'),
            '10': ('135:12:0','45:36:0'),
            '11': ('123:42:0','90:24:0'),
            '12': ('183:42:0','90:18:0')}

fermi = Spacecraft(gbm_pointings,window=0.1)

res = 250
rr,dd = np.meshgrid(np.linspace(0,360,res,endpoint=False),np.linspace(-90,90,res))
exposure_positions = np.vstack([rr.ravel(),dd.ravel()])
gbm_exposures = np.array([[ detector.exposure(position[0],position[1]) for position in exposure_positions.T] 
                      for detector in fermi.detectors])

In [10]:
#using SkyCoord to convert coordinates to degrees and solve for distances.

def separation(ra1,dec1,ra2,dec2):

    c=SkyCoord(ra=ra1*u.deg,dec=dec1*u.deg)
    d=SkyCoord(ra=ra2*u.deg,dec=dec2*u.deg)
    dist=c.separation(d)
    dist=dist.value

    return dist

In [30]:
def throw_grbs(fermi,minflux,maxflux,pf): #,detect):
    
    nsims=int(np.round(len(pf)*2))
    ra,dec=random_sky(nsims)
    ra=np.array(ra)-180
    dec=np.array(dec)
    #sigma=0.65,mean=1.5
 
    #change the sigma and mean in order to create a log fit for simulated GBM. Automate this fit.
    flux=np.random.lognormal(size=nsims,sigma=0.5,mean=0.55)*(np.log10(maxflux)-np.log10(minflux))+np.log10(minflux)

    #GBM exposures for each random GRB. Believe this is an array with the different exposures for each detector
    randgbmexposures = np.array([[detector.exposure(ra[i],dec[i]) for i in range(nsims)] for detector in fermi.detectors], dtype=float)
   # print(randgbmexposures)
   # randgbmexposures = np.random.rand((17,22), dtype=float)
    numrows = len(randgbmexposures) 
   # randgbmex = np.lexsort(randgbmexposures[...,...])

   # print(randgbmexposures)
    #Order randgbmexposures into descending order
    #randgbmex = -np.sort(-randgbmexposures, axis =1) 
    print(numrows)


    
#Find highest number in each axis of the array and remove it
 
    
#    BCsens = minflux/Aratio/np.array(randexposures.max(axis=0))
   # GBMsens = minflux/np.array(randgbmexposures.max(axis=0))

#     inside=np.where(separation(ra,dec,0,35)<100.)[0]
#     ninside=len(inside)
#     print ninside
#    w=np.where(flux-BCsens>0)[0]
#    bcdetfrac=float(len(w))/nsims

#    wg=np.where(flux-GBMsens>0)[0]
#    gbmdetfrac=float(len(wg))/nsims

        
    return randgbmexposures


In [31]:
flux=gbm['FLUX_BATSE_64'][s]
minflux=min(flux)
maxflux=max(flux)

throw_grbs(fermi,minflux,maxflux,pf)

12


array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.61694463,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.28422566,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.70546651,
        0.        ]])